# Clustering Crypto

In [ ]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [ ]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [ ]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.

In [ ]:
# Alternatively, use the provided csv file:
from pathlib import Path
file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame
crypto_data2 = pd.read_csv(file_path)
crypto_data2.head()

### Data Preprocessing

In [ ]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'

crypto_data2.rename(columns = {'Unnamed: 0':'id'}, inplace = True)
crypto_df = crypto_data2.set_index('id')
crypto_df.head()

In [ ]:
# Keep only cryptocurrencies that are trading
crypto_trading = crypto_df[crypto_df['IsTrading']==True]
crypto_trading.head()

In [ ]:
# Keep only cryptocurrencies with a working algorithm
crypto_trading = crypto_df[crypto_df['Algorithm']!="NaN"]
crypto_trading.head()

In [ ]:
# Remove the "IsTrading" column
crypto_trading.drop(columns='IsTrading', inplace=True)
crypto_trading.head()

In [ ]:
# Remove rows with at least 1 null value
crypto_trading.dropna(inplace=True)
crypto_trading.head()

In [ ]:
# Remove rows with cryptocurrencies having no coins mined
crypto_trading = crypto_trading[crypto_trading['TotalCoinsMined']!=0]
crypto_trading.head()

In [ ]:
# Drop rows where there are 'N/A' text values
crypto_trading = crypto_trading[crypto_trading['CoinName']!='N/A']
crypto_trading = crypto_trading[crypto_trading['Algorithm']!='N/A']
crypto_trading = crypto_trading[crypto_trading['ProofType']!='N/A']
crypto_trading.head()

In [ ]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coinname = pd.DataFrame(crypto_trading['CoinName'])
coinname.head()

In [ ]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_trading.drop(columns='CoinName', inplace=True)
crypto_trading.head()

In [ ]:
# Create dummy variables for text features
X = pd.get_dummies(crypto_trading, columns=['Algorithm','ProofType'])
X.head()

In [ ]:
# Standardize data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### Reducing Dimensions Using PCA

In [ ]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA()
x_pca = pca.fit_transform(X_scaled)
exp_var_pca = pca.explained_variance_ratio_

pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)
X_pca

In [ ]:
# Create a DataFrame with the principal components data
pcs_df = pd.DataFrame(
    data=X_pca, columns=["PC 1", "PC 2","PC 3"], index = crypto_trading.index
)
pcs_df.head()

### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [ ]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)


Running K-Means with `k=<your best value for k here>`

In [ ]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)
# Fit the model
model.fit(pcs_df)
# Predict clusters
predictions = model.predict(pcs_df)
# Create a new DataFrame including predicted clusters and cryptocurrencies features
pcs_df["class"] = model.labels_
clustered_df = pd.concat([coinname,crypto_trading, pcs_df], axis =1, join='inner')

#maintain columns: "Algorithm", "ProofType", "TotalCoinsMined", "TotalCoinSupply", "PC 1", "PC 2", "PC 3", "CoinName", "Class"
clustered_df.head()

### Visualizing Results

#### 3D-Scatter with Clusters

In [ ]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="class",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

#### Table of Tradable Cryptocurrencies

In [ ]:
# Table with tradable cryptos
clustered_df.hvplot.table(columns=["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "class"], sortable=True, selectable=True)

In [ ]:
# Print the total number of tradable cryptocurrencies
print(f'Total number of tradable cyptocurrencies is {len(clustered_df)}')

#### Scatter Plot with Tradable Cryptocurrencies

In [ ]:
# Scale data to create the scatter plot
minmax_scale = MinMaxScaler()
coin_df = clustered_df.loc[:,'TotalCoinsMined':'TotalCoinSupply']
coin_df.head()
clustered_scaled = pd.DataFrame(minmax_scale.fit_transform(coin_df), columns=['TotalCoinsMined','TotalCoinSupply'], index = clustered_df['CoinName'])
clustered_scaled.reset_index(inplace=True)
clustered_scaled.head()

In [ ]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
clustered_scaled.hvplot.scatter(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    hover_cols='CoinName',
)